In [295]:
import numpy as np
import matplotlib.pyplot as plt
import ipycanvas as ipc
from ipycanvas import Canvas, hold_canvas
def drawField(canvas):
# Display the canvas
    canvas.stroke_style = 'green'
#   Draw the outer rectangle representing the soccer field
    canvas.stroke_rect(50, 50, 700, 400)
    canvas.stroke_style = 'green'
# Draw the center line dividing the field into two halves
    canvas.begin_path()
    canvas.move_to(400, 50)
    canvas.line_to(400, 450)
    canvas.stroke()
# Draw the center circle with a radius of 50 pixels
    canvas.begin_path()
    canvas.arc(400, 250, 50, 0, 2 * np.pi)
    canvas.stroke()
    canvas.stroke_style = 'green'
# Draw the left penalty area
    canvas.stroke_rect(50, 150, 100, 200)
# Draw the right penalty area
    canvas.stroke_rect(650, 150, 100, 200)
# Draw the left penalty arc
    canvas.begin_path()
    canvas.arc(150, 250, 50, 1.5 * np.pi, 0.5 * np.pi)
    canvas.stroke()
# Draw the right penalty arc
    canvas.begin_path()
    canvas.arc(650, 250, 50, 0.5 * np.pi, 1.5 * np.pi)
    canvas.stroke()
# Draw goals
    canvas.stroke_style = 'maroon'
# Draw the left goal
    canvas.stroke_rect(30, 200, 20, 100)
# Draw the right goal
    canvas.stroke_rect(750, 200, 20, 100)
    return canvas

#draw player in right half using a filled circle
def draw_player(canvas,x_player,y_player,has_ball=False,color='red'):
    canvas.fill_style = color
    canvas.fill_arc(x_player, y_player, 10, 0, 2 * np.pi)
    if has_ball:
        canvas.fill_style='black'
        canvas.fill_arc(x_player+5, y_player, 3, 0, 2 * np.pi)
def draw_ball(canvas,ball):
    canvas.fill_style = 'black'
    if ball.with_player==False:
        x_ball=ball.x
        y_ball=ball.y
        canvas.fill_arc(x_ball, y_ball, 5, 0, 2 * np.pi)

def set_random_positions():
    x_player_with_ball=400+np.random.randint(10, 150)   
    y_player_with_ball=50+np.random.randint(50, 150) 
    x_player_without_ball=400+np.random.randint(10, 150)
    y_player_without_ball=50+np.random.randint(250, 350)
    x_defender=400+np.random.randint(170, 220)
    y_defender=50+np.random.randint(125, 275)
    v_attackers=5 # pixels per frame
    v_defender=5 # pixels per frame
    v_ball=15 # pixels per frame  
    return x_player_with_ball,y_player_with_ball,x_player_without_ball,y_player_without_ball,x_defender,y_defender,v_attackers,v_defender,v_ball


In [ ]:
canvas=Canvas(width=800, height=500)
display(canvas)
class Player:
    def __init__(self,x,y,has_ball=False,role='attacker',side='center',x_target=750,y_target=250):
        self.x=x
        self.y=y
        self.x_target=x_target
        self.y_target=y_target
        self.has_ball=has_ball
        self.direction=[-999.,-999.]
        self.speed_max=5
        self.role=role
        self.side=side
        if role=='defender':
            self.color='blue'
        else:
            self.color='red'
        if role=='defending_goalie':
            self.color='purple'
    def move(self,dx,dy):
        self.x+=dx
        self.y+=dy
    def get_position(self):
        return self.x,self.y
    def get_has_ball(self):
        return self.has_ball
    def set_has_ball(self,has_ball):
        self.has_ball=has_ball

class Ball:
    def __init__(self,x,y,with_player=True,x_dest=750,y_dest=250):
        self.x=x
        self.y=y
        self.x_dest=x_dest
        self.y_dest=y_dest
        self.direction=[-999.,-999.]
        self.with_player=with_player
        self.speed_max=14
    def move(self,dx,dy):
        self.x+=dx
        self.y+=dy

x_player_with_ball,y_player_with_ball,x_player_without_ball,y_player_without_ball,x_defender,y_defender,v_attackers,v_defender,v_ball=set_random_positions()
attackers=[Player(x_player_with_ball,y_player_with_ball,True,side='left',x_target=750,y_target=240),\
        Player(x_player_without_ball,y_player_without_ball,side='right',x_target=750,y_target=260)]
defenders=[Player(x_defender,y_defender,False,'defender'),\
           Player(740,250,False,'defending_goalie')]
ball=Ball(x_player_with_ball,y_player_with_ball)
def distance_point_to_line(x, y, x0,y0, dx, dy):
    eps=1e-3
    a=-dy
    b=dx
    c=-(a*x0+b*y0)
    # Calculate the distance using the formula
    distance = np.abs(a * x + b * y+c) / np.sqrt(a**2 + b**2+eps)
    return distance
global waiting_player
def game_dt(attackers,defenders,ball,dt=1,d_min=25):
    x_max=0
    eps=1e-3
    global waiting_player
    # determine the x_max of defenders
    for player in defenders:
        if player.role=='defender':
            if player.x>x_max:
                x_max=player.x
                x_defender=player.x
                y_defender=player.y
    if ball.with_player==False:
        dx=ball.x_dest-ball.x
        dy=ball.y_dest-ball.y
        d_tot=np.sqrt(dx**2+dy**2)
        vx=dx/np.sqrt(dx**2+dy**2+eps)*ball.speed_max*dt
        vy=dy/np.sqrt(dx**2+dy**2+eps)*ball.speed_max*dt
        if(np.sqrt(vx**2+vy**2)>d_tot):
            v_fraction=d_tot/np.sqrt(vx**2+vy**2)
        else:
            v_fraction=1
        ball.move(vx*v_fraction,vy*v_fraction)     
        dist_2_waiting=np.sqrt((attackers[waiting_player].x-ball.x)**2+(attackers[waiting_player].y-ball.y)**2)
        if dist_2_waiting<5:
            ball.with_player=True
            attackers[waiting_player].has_ball=True
            ball.x_dest=attackers[waiting_player].x_target
            ball.y_dest=attackers[waiting_player].y_target
            return
    if ball.with_player:
        for player in attackers:
            if player.role=='attacker':
                if player.has_ball:
                    dx=player.x_target-player.x
                    dy=player.y_target-player.y
                    vx=dx/np.sqrt(dx**2+dy**2)*player.speed_max*dt
                    vy=dy/np.sqrt(dx**2+dy**2)*player.speed_max*dt
                    player.move(vx,vy)
                    ball.x=player.x
                    ball.y=player.y
                    ball.x_dest=player.x_target
                    ball.y_dest=player.y_target
                    dist_2_defender=np.sqrt((x_defender-player.x)**2+(y_defender-player.y)**2)
                    if dist_2_defender<d_min:
                        for i_wait,other_player in enumerate(attackers):
                            if other_player.role=='attacker' and other_player.has_ball==False:
                                waiting_player=i_wait
                        player.has_ball=False
                        ball.with_player=False
                        ball.x_dest=other_player.x
                        ball.y_dest=other_player.y
                else:
                    dx=player.x_target-player.x
                    dy=player.y_target-player.y
                    vx=dx/np.sqrt(dx**2+dy**2)*player.speed_max*dt
                    vy=dy/np.sqrt(dx**2+dy**2)*player.speed_max*dt
                    if player.x+vx>x_max:
                        v_fraction=(x_max-player.x)/vx
                    else:
                        v_fraction=1
                    player.move(vx*v_fraction,vy*v_fraction)
        for player in defenders:
            if player.role=='defender':
                dx=ball.x_dest-ball.x
                dy=ball.y_dest-ball.y
                dx1=dy
                dy1=-dx
                vx=dx1/np.sqrt(dx1**2+dy1**2+eps)*player.speed_max*dt
                vy=dy1/np.sqrt(dx1**2+dy1**2+eps)*player.speed_max*dt
                x_new=player.x+vx
                y_new=player.y+vy
                d_old=np.sqrt((player.x-ball.x)**2+(player.y-ball.y)**2)
                d_new=np.sqrt((x_new-ball.x)**2+(y_new-ball.y)**2)
                dist_to_intercept=distance_point_to_line(player.x,player.y,ball.x,ball.y,dx,dy)
                if dist_to_intercept<5:
                    v_fraction=0#dist_to_intercep
                else:
                    v_fraction=1
                if d_new<d_old:
                    player.move(vx*v_fraction,vy*v_fraction)
                else:
                    player.move(-vx*v_fraction,-vy*v_fraction)
    
from time import sleep
for i in range(80):
    game_dt(attackers,defenders,ball)
    with hold_canvas():
        canvas.clear()
        drawField(canvas)
        for player in attackers:
            draw_player(canvas,player.x,player.y,player.has_ball,player.color)
        for player in defenders:
            draw_player(canvas,player.x,player.y,player.has_ball,player.color)
        draw_ball(canvas,ball)
        sleep(0.02)
#canvas


Canvas(width=800)

697.5687418847299 288.32224210690936
750 240
